# Functions, Methods and Macros



## Functions

Python functions supports positional arguments and default values:

In [1]:
def func_default_arg(x, y=1):
    print(f"{x} , {y}")

However, unlike in Python, positional arguments must not be named when the function is called:

In [2]:
func_default_arg(1, y=1) # no error 

1 , 1


Python also supports a variable number of arguments (called "varargs") using the syntax `*arg`, which is the equivalent of Julia's `arg...`:

In [3]:
def func_multiple_arg(*myarglist):
    print(f"multiple argument list: {myarglist}")

func_multiple_arg("first string", "apple", "orange")

multiple argument list: ('first string', 'apple', 'orange')


Keyword arguments are supported:

In [4]:
def copy_files2(*paths, confirm=False, target_dir):
    print(f"paths={paths}, confirm={confirm}, {target_dir}") # string interpolation

copy_files2("a.txt", "b.txt", target_dir="/tmp")

paths=('a.txt', 'b.txt'), confirm=False, /tmp


Notes:
* `target_dir` has no default value, so it is a required argument.
* The order of the keyword arguments does not matter.

You can have another vararg in the keyword section. It corresponds to Python's `**kwargs`:

In [5]:
def copy_files3(*paths, confirm=False, target_dir, **options):
    print(f"paths={paths}, confirm={confirm}, {target_dir}")
    verbose = options["verbose"]
    print(f"verbose={verbose}")

copy_files3("a.txt", "b.txt", target_dir="/tmp", verbose=True, timeout=60)

paths=('a.txt', 'b.txt'), confirm=False, /tmp
verbose=True


In [8]:
def foo1(a, b=2, c=3, /):
    print(f"a: {a}, b: {b}, c: {c}")

foo1(1, 2)

def foo2(a, b=2, /, *c):
    print(f"a: {a}, b: {b}, c: {c}")

foo2(1, 2, 3, 4)

a: 1, b: 2, c: 3
a: 1, b: 2, c: (3, 4)


|Julia|Python (3.8+ if `/` is used)
|-----|------
| `function foo(a, b=2, c=3)`<br />&nbsp;&nbsp;&nbsp;&nbsp;`...`<br />`end`<br /><br />`foo(1, 2) # positional only` | `def foo(a, b=2, c=3, /):`<br />&nbsp;&nbsp;&nbsp;&nbsp;`...`<br /><br />`foo(1, 2) # pos only because of /`
| `function foo(;a=1, b, c=3)`<br />&nbsp;&nbsp;&nbsp;&nbsp;`...`<br />`end`<br /><br />`foo(c=30, b=2) # keyword only` | `def foo(*, a=1, b, c=3):`<br />&nbsp;&nbsp;&nbsp;&nbsp;`...`<br /><br />`foo(c=30, b=2) # kw only because of *`
| `function foo(a, b=2; c=3, d)`<br />&nbsp;&nbsp;&nbsp;&nbsp;`...`<br />`end`<br /><br />`foo(1; d=4) # pos only; then keyword only` | `def foo(a, b=2, /, *, c=3, d):`<br />&nbsp;&nbsp;&nbsp;&nbsp;`...`<br /><br />`foo(1, d=4) # pos only then kw only`
| `function foo(a, b=2, c...)`<br />&nbsp;&nbsp;&nbsp;&nbsp;`...`<br />`end`<br /><br />`foo(1, 2, 3, 4) # positional only` | `def foo(a, b=2, /, *c):`<br />&nbsp;&nbsp;&nbsp;&nbsp;`...`<br /><br />`foo(1, 2, 3, 4) # positional only`
| `function foo(a, b=1, c...; d=1, e, f...)`<br />&nbsp;&nbsp;&nbsp;&nbsp;`...`<br />`end`<br /><br />`foo(1, 2, 3, 4, e=5, x=10, y=20)`<br /> | `def foo(a, b=1, /, *c, d=1, e, **f):`<br />&nbsp;&nbsp;&nbsp;&nbsp;`...`<br /><br />`foo(1, 2, 3, 4, e=5, x=10, y=20)`


## Concise Functions

In Python, the following definition:

In [9]:
increase_by_one = lambda x: x + 1

is equivalent to:

In [10]:
def increase_by_one(x):
    x + 1

For example, here's a shorter way to define the `π_approx4()` function in Julia:

\begin{equation}
\pi = \sqrt{12} \sum_0^\infty {\frac{(\frac{-1}{3})^i}{2i+1} }
\end{equation}

In [11]:
isodd = lambda n: n % 2 == 1
pi_approx = lambda n: 12**0.5 * sum([(-1 if isodd(i) else 1) * (1/3)**i / (2*i+1) for i in range(0, n+1)]) 


In [12]:
import math
print(pi_approx(100)- math.pi)

8.881784197001252e-16


## Anonymous Functions
Just like in Python, you can define anonymous functions:

In [15]:
u = lambda x: x**2
list(map(u, range(1, 5)))

[1, 4, 9, 16]

Here is the equivalent Python code:

```python
list(map(lambda x: x**2, range(1, 5)))
```

Notes:
* `map()` returns an array in Julia, instead of an iterator like in Python.
* You could use a comprehension instead: `[x^2 for x in 1:4]`.


|Julia|Python
|-----|------
|`x -> x^2` | `lambda x: x**2`
|`(x,y) -> x + y` | `lambda x,y: x + y `
|`() -> println("yes")` | `lambda: print("yes")`


In Python, lambda functions must be simple expressions. They cannot contain multiple statements. 

## Piping
If you are used to the Object Oriented syntax `"a b c".upper().split()`, you may feel that writing `split(uppercase("a b c"))` is a bit backwards. If so, the piping operation `|>` is for you:

In [16]:
"a b c".upper().split()


['A', 'B', 'C']

If you want to pass more than one argument to some of the functions, you can use anonymous functions:

In [17]:
", ".join("a b c".upper().split())

'A, B, C'

## Composition

Python does not have this feature in this way.

## Estimating π and ℯ

Let's write our first function. It will estimate π using the equation:
\begin{equation}
\pi=\sqrt{12} \sum_{k=0}^{\infty} \frac{\left(-\frac{1}{3}\right)^{k}}{2 k+1}=\sqrt{12}\left(1-\frac{1}{3 \cdot 3}+\frac{1}{5 \cdot 3^{2}}-\frac{1}{7 \cdot 3^{3}}+\cdots\right)
\end{equation}

In [18]:
def π_approx(n):
    sum_ = 1.0
    for i in range(1, n+1):
        sum_ += (-1 if isodd(i) else 1) * (1/3)**i / (2*i + 1)

    return 12.0**0.5 * sum_

p = π_approx(100_000_000)
print(f"π ≈ {p}")
print(f"Error is {(p - math.pi)}")

π ≈ 3.141592653589794
Error is 8.881784197001252e-16


In [20]:
pi_approx_inline = lambda n: 12.0**0.5 * sum([(-1 if isodd(i) else 1) * (1/3)**i / (2*i + 1) for i in range(0, n+1)])


Pretty similar, right? But notice the small differences:

|Julia|Python
|-----|------
|`function` | `def`
|`for i in X`<br />&nbsp;&nbsp;&nbsp;&nbsp;`...`<br />`end` | `for i in X:`<br />&nbsp;&nbsp;&nbsp;&nbsp;`...`
|`1:n` | `range(1, n+1)`
|`cond ? a : b` | `a if cond else b`
|`2i + 1` | `2 * i + 1`
|`4s` | `return 4 * s`
|`println(a, b)` | `print(a, b, sep="")`
|`print(a, b)` | `print(a, b, sep="", end="")`
|`"$p"` | `f"{p}"`
|`"$(p - π)"` | `f"{p - math.pi}"`

This example shows that:
* Julia can be just as concise and readable as Python.
* Indentation in Julia is _not_ meaningful like it is in Python. Instead, blocks end with `end`.
* Many math features are built in Julia and need no imports.
* There's some mathy syntactic sugar, such as `2i` (but you can write `2 * i` if you prefer).
* In Julia, the `return` keyword is optional at the end of a function. The result of the last expression is returned (`4s` in this example).
* Julia loves Unicode and does not hesitate to use Unicode characters like `π`. However, there are generally plain-ASCII equivalents (e.g., `π == pi`).

Now let's compute the Euler's number
$$
e=\sum_{n=0}^{\infty} \frac{1}{n !}=1+\frac{1}{1}+\frac{1}{1 \cdot 2}+\frac{1}{1 \cdot 2 \cdot 3}+\cdots
$$

In [21]:
def e_approx1(n):
    sum_ = 1.0
    factorial_ = 1.0
    for i in range(1, n+1):
        factorial_ *= i
        sum_ += 1 / factorial_

    return sum_

In [22]:
def factorial(n):
    if n == 0 or n == 1:
        return 1

    return factorial(n-1) * n

def e_approx2(n):
    sum_ = 0.0
    for i in range(0, n+1): 
        sum_ += 1 / factorial(i)

    return sum_

In [23]:
import time

s = time.time()
e_approx1(100)
e = time.time()

print(f"time: {e - s}")

error1 = e_approx1(100) - math.e
print(error1)

s = time.time()
e_approx1(100)
e = time.time()

print(f"time: {e - s}")

error2 = e_approx1(100) - math.e
print(error2)


time: 0.0
4.440892098500626e-16
time: 0.0
4.440892098500626e-16


In [24]:
e_approx_inline = lambda n: sum(1/factorial(i) for i in range(0, n+1))

e_approx_inline(100) - math.e

4.440892098500626e-16

## Loop Fusion
Did you notice that we wrote `sin.(x) ./ x` (like `sin(x) / x` in numpy)?

In [25]:
import numpy as np
from math import sin

x = np.array([1, 2, 3, 4, 5])
a = np.sin(x) / x
b = [sin(i) / i for i in x]
assert (a == b).all()

 This is not just syntactic sugar: it's actually a very powerful Julia feature. Indeed, notice that the array only gets traversed once. Even if we chained more than two dotted operations, the array would still only get traversed once. This is called _loop fusion_.

In contrast, when using NumPy arrays, `sin(x) / x` first computes a temporary array containing `sin(x)` and then it computes the final array. Two loops and two arrays instead of one. NumPy is implemented in C, and has been heavily optimized, but if you chain many operations, it still ends up being slower and using more RAM than Julia.


## Python is slow!
Let's compare the Julia and Python implementations of the `π_approx()` function:

In [26]:
import time

s = time.time()
pi_approx(100_000_000)
e = time.time()

print(f"time: {e - s}")

time: 52.61412715911865


It looks like Julia is close to 100 times faster than Python in this case! To be fair, `PyCall` does add some overhead, but even if you run this code in a separate Python shell, you will see that Julia crushes (pure) Python when it comes to speed.

So why is Julia so much faster than Python? Well, **Julia compiles the code on the fly as it runs it**.

Okay, let's summarize what we learned so far: Julia is a dynamic language that looks and feels a lot like Python, you can even execute Python code super easily, and pure Julia code runs much faster than pure Python code, because it is compiled on the fly. I hope this convinces you to read on!

Next, let's continue to see how Python's main constructs can be implemented in Julia.

# Methods
Earlier, we discussed structs, which look a lot like Python classes, with instance variables and constructors, but they did not contain any methods (just the inner constructors). In Julia, methods are defined separately, like regular functions:

In [28]:
class Option:

    def __init__(self, iscall, strike_price, maturity) -> None:
        self.iscall, self.strike_price, self.maturity = iscall, strike_price, maturity


option = Option(True, 100.0, 1.0)
option.iscall

True

In [29]:
def payoff(option, price):
    return max(price - option.strike_price, 0.0) if option.iscall else max(option.strike_price - price , 0.0)  

put = Option(False, 100.0, 1.0) # constructor of Option
payoff(put, 90.0) 

10.0

Since the `payoff()` method in Julia is not bound to any particular type, we can use it with any other type we want, as long as that type has a `iscall` and an `strike_price`:

In [30]:
class BarrierOption:

    def __init__(self, iscall, strike_price, maturity, isknockin, lower_barrier, upper_barrier) -> None:
        self.iscall, self.strike_price, self.maturity = iscall, strike_price, maturity
        self.isknockin, self.lower_barrier, self.upper_barrier = isknockin, lower_barrier, upper_barrier


barrier_option = BarrierOption(True, 100.0, 1.0, True, 80.0, 90.0)
payoff(barrier_option, 130.0)

30.0

## Extending a Function
One nice thing about having a class hierarchy is that you can override methods in subclasses to get specialized behavior for each class. For example, in Python you could override the `payoff()` method like this:


In [ ]:
# A is B => A is a subtype of B 

In [31]:
class Option:
    def __init__(self, iscall, strike_price, maturity) -> None:
        self.iscall, self.strike_price, self.maturity = iscall, strike_price, maturity

    def payoff(self, price):
        return max(price - self.strike_price, 0.0) if self.iscall else max(self.strike_price - price , 0.0)            

class BinaryOption(Option):
    def __init__(self, iscall, strike_price, maturity) -> None:
        super().__init__(iscall, strike_price, maturity)
        

    def payoff(self, price):
        call_payoff = 1 if price > self.strike_price else 0
        put_payoff = 1 if self.strike_price > price else 0
        return call_payoff if self.iscall else put_payoff  

    def isexercised(self, price):
        call_exercised = price > self.strike_price # output type Boolean
        put_exercised = price < self.strike_price # output type Boolean
        return call_exercised if self.iscall else put_exercised # output type Boolean

barrier_option = BinaryOption(False, 100.0, 1.0)

print(barrier_option.payoff(90.0))

barrier_option.isexercised(110.0)

1


False

Notice that the expression `d.payoff()` will call a different method if `d` is an `Option` or a `BarrierOption`. This is called "polymorphism": the same method call behaves differently depending on the type of the object. The language chooses which actual method implementation to call, based on the type of `d`: this is called method "dispatch". More specifically, since it only depends on a single variable, it is called "single dispatch".

The good news is that Julia can do single dispatch as well:

## Multiple Dispatch

Python is single dispatch.

## Abstract Types

Python does not have this feature in this way.

## Iterator Interface
You will sometimes want to provide a way to iterate over your custom types. In Python, this requires defining the `__iter__()` method which should return an object which implements the `__next__()` method. In Julia, you must define at least two functions:
* `iterate(::YourIteratorType)`, which must return either `nothing` if there are no values in the sequence, or `(first_value, iterator_state)`.
* `iterate(::YourIteratorType, state)`, which must return either `nothing` if there are no more values, or `(next_value, new_iterator_state)`.

For example, let's create a simple iterator for the Fibonacci sequence:

In [39]:
class Fibonacci():
    def __init__(self, n):
        self.counter = n
        self.current  = 0
        self.next = 1

    def __iter__(self):
        return self

    def __next__(self):

        if self.counter == 0:
           raise StopIteration

        self.counter -= 1

        next = self.current + self.next
        self.current = self.next
        self.next = next

        return self.current

Now we can iterate over a `Fibonacci` instance:

In [42]:
fib_10 = Fibonacci(10)
for i, fib in enumerate(fib_10):
    print(f"{i + 1}: {fib}")

1: 1
2: 1
3: 2
4: 3
5: 5
6: 8
7: 13
8: 21
9: 34
10: 55


## Indexing Interface


In [23]:
class LinkedList:
    def __init__(self, n) -> None:
        self.current = n
        if n != 0:
            self.previous_block = LinkedList(n - 1)

    def print_list(self):
        print(self.current)
        try:
            self.previous_block.print_list()
        except:
            pass

    def __getitem__(self, index):
        current_block = self
        for i in range(index):
            current_block = current_block.previous_block

        return current_block.current

    def __setitem__(self, index, new_value):
        current_block = self
        for i in range(index):
            current_block = current_block.previous_block

        current_block.current = new_value


ll = LinkedList(5)
ll.print_list()

print(ll[3])
ll[3] = 8
print(ll[3])



5
4
3
2
1
0
2
8


## Creating a Number Type

Python does not have this feature in this way. (only support operator overloading)


In [48]:
class Rational:

    def __init__(self, num, den) -> None:
        self.num, self.den = num, den

    def __add__(self, other):
        return Rational(self.num * other.den + self.den * other.num, self.den * other.den)

    def __sub__(self, other):
        return Rational(self.num * other.den - self.den * other.num, self.den * other.den)

    def __mul__(self, other):
        return Rational(self.num * other.num, self.den * other.den)

    def __str__(self) -> str:
        return f"{self.num} / {self.den}"

r1 = Rational(1, 2)
r2 = Rational(1, 3)

print(r1 + r2)
print(r1 - r2)
print(r1 * r2)

5 / 6
1 / 6
1 / 6


## Conversion

Python does not have this feature in this way.

## Promotion

Python does not have this feature in this way.

## Parametric Types and Functions

Python does not have this feature in this way.


# Docstrings
It's good practice to add docstrings to every function you export. The docstring is placed just _before_ the definition of the function:

In [55]:
def squere(x):
    """
    Compute the square of number x
    """   
    return x**2

    

You can retrieve a function's docstring using the `@doc` macro:

In [57]:
print(squere.__doc__)


    Compute the square of number x
    
